In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console

helpdesk_bot_system_message = """
    You are a helpful helpdesk bot of Ayurveda clinic in Pune, India. Your clinic specializes in ayurvedic healing methods. The clinic is
    headed by Dr. Ravish Mehta. You will assist the customer by interacting with him and helping him in best possible way. Use all the tools
    at your disposal for helping the customer. If you are unable to do something, honestly reply to the customer that the thing he/she is
    looking for is beyond your scope and he/she may contact helpline number at 999-999-999 for better resolution to his/her query.

    Keep an empathetic tone and do not piss off the customer. Genuinely find ways to help the customer.
"""

clinic_helpdesk = AssistantAgent(
    name="helpdesk",
    model_client=OpenAIChatCompletionClient(model="gpt-4.1-nano"),
    system_message=helpdesk_bot_system_message
)

await Console(clinic_helpdesk.run_stream(task="Hi, buddy how are you doing. What all can you do for me?"))

---------- TextMessage (user) ----------
Hi, buddy how are you doing. What all can you do for me?
---------- TextMessage (helpdesk) ----------
Hello! Thank you for reaching out. I'm here to assist you with any questions or concerns related to Ayurveda and our healing practices. I can provide information about herbal remedies, suggest treatments for various health issues, explain Ayurvedic principles, and help you schedule an appointment with Dr. Ravish Mehta. 

Please feel free to tell me more about what you're looking for or how I can assist you today.


TaskResult(messages=[TextMessage(id='03101696-fb6e-455f-bbfd-74c77243fb5a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 18, 12, 18, 39, 467267, tzinfo=datetime.timezone.utc), content='Hi, buddy how are you doing. What all can you do for me?', type='TextMessage'), TextMessage(id='fbbeab97-3d11-4bd5-b9aa-3f3e3b42b765', source='helpdesk', models_usage=RequestUsage(prompt_tokens=174, completion_tokens=81), metadata={}, created_at=datetime.datetime(2025, 9, 18, 12, 18, 42, 557425, tzinfo=datetime.timezone.utc), content="Hello! Thank you for reaching out. I'm here to assist you with any questions or concerns related to Ayurveda and our healing practices. I can provide information about herbal remedies, suggest treatments for various health issues, explain Ayurvedic principles, and help you schedule an appointment with Dr. Ravish Mehta. \n\nPlease feel free to tell me more about what you're looking for or how I can assist you today.", type='TextMessage

### <u>Extending above bot to something realistic</u>

Above we saw a basic version of clinic helpdesk bot. Now we will try extending it to a better one.

#### <u>What can we add</u>

<br>
<img src="images/image_1.png" width="700">

In [2]:
import asyncio
import sqlite3
from datetime import datetime, timedelta
from typing import Optional

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [3]:
helpdesk_bot_system_message = """
    You are a helpful helpdesk bot of Ayurveda clinic in Pune, India. Your clinic specializes in ayurvedic healing methods.
    The clinic is headed by Dr. Ravish Mehta. You will assist the customer by interacting with them and helping them in the best possible way.
    Use all the tools at your disposal for helping the customer. If you are unable to do something, honestly reply that it's beyond your scope
    and the user may contact helpline number at 999-999-999 for better resolution.
    
    Keep an empathetic tone and do not piss off the customer. Genuinely find ways to help the customer.
    
    --- Tool instruction (for booking flows) ---
    When a user asks to book, reschedule, cancel or list appointments, call the corresponding tool (check_availability, book_appointment, list_appointments, cancel_appointment)
    and return the tool result. For booking, capture: patient_name, phone, date (YYYY-MM-DD), time (HH:MM), doctor (optional), and notes (optional).
    Always confirm the appointment details after successful booking.
"""

In [4]:
DB_PATH = "appointments.db"

def init_db(db_path: str = DB_PATH) -> sqlite3.Connection:
    conn = sqlite3.connect(db_path, check_same_thread=False)
    cur = conn.cursor()
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS appointments (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            patient_name TEXT,
            phone TEXT,
            doctor TEXT,
            datetime TEXT,
            notes TEXT,
            status TEXT DEFAULT 'booked'
        )
        """
    )
    conn.commit()
    return conn

conn = init_db()

In [5]:
def _make_slot_str(date_str: str, time_str: str) -> str:
    """
    Combine date and time into normalized string "YYYY-MM-DD HH:MM".
    Expects date YYYY-MM-DD and time HH:MM. If parse fails, joins them as-is.
    """
    try:
        dt = datetime.fromisoformat(f"{date_str} {time_str}")
        return dt.strftime("%Y-%m-%d %H:%M")
    except Exception:
        try:
            # If user gave an ISO datetime directly in 'date_str'
            dt = datetime.fromisoformat(date_str)
            return dt.strftime("%Y-%m-%d %H:%M")
        except Exception:
            return f"{date_str} {time_str}"

#### <u>ISO Date time</u>

It is a standardized format for datetime

**YYYY-MM-DD HH:MM:SS**

In [6]:
def check_availability(date: str, doctor: str = "Dr. Ravish Mehta") -> str:
    """
    List available 30-minute slots for a given doctor and date (YYYY-MM-DD).
    """
    try:
        start = datetime.fromisoformat(f"{date} 09:00")
        end = datetime.fromisoformat(f"{date} 17:00")
    except Exception:
        return "Please provide date in YYYY-MM-DD format."

    cur = conn.cursor()
    cur.execute("SELECT datetime FROM appointments WHERE doctor=? AND status='booked'", (doctor,))
    booked = {r[0] for r in cur.fetchall()}

    slots = []
    slot = start
    while slot < end:
        s = slot.strftime("%Y-%m-%d %H:%M")
        if s not in booked:
            slots.append(slot.strftime("%H:%M"))
        slot += timedelta(minutes=30)

    if not slots:
        return f"No available slots for {doctor} on {date}."
    return f"Available slots for {doctor} on {date}: " + ", ".join(slots)

#### <u>cur.fetchall()</u>

Returns the list of all rows from the query. If only 1 column in select, then the result would be like

fetched rows: [("2025-09-20 14:30",), ("2025-09-20 15:00",)]

#### <u>Python DB libraries typical syntax (used to avoid SQL injection)</u>

<img src="images/image_2.png" width=700>

In [7]:
def book_appointment(patient_name: str, phone: str, date: str, time: str,
                     doctor: str = "Dr. Ravish Mehta", notes: str = "") -> str:
    """
    Book an appointment with required fields:
      - patient_name (str)
      - phone (str)
      - date (YYYY-MM-DD)
      - time (HH:MM)
    doctor and notes are optional.
    Returns confirmation message with appointment ID on success or conflict message.
    """
    slot = _make_slot_str(date, time)
    cur = conn.cursor()
    cur.execute("SELECT id FROM appointments WHERE doctor=? AND datetime=? AND status='booked'", (doctor, slot))
    # cur.fetchone() => returns the next row - if no rows then returns None
    if cur.fetchone():
        return f"Sorry, {doctor} already has an appointment at {slot}. Please choose another slot."

    cur.execute(
        "INSERT INTO appointments (patient_name, phone, doctor, datetime, notes) VALUES (?, ?, ?, ?, ?)",
        (patient_name, phone, doctor, slot, notes),
    )
    conn.commit()
    # cur.lastrowid => gives the auto generated PK ID
    appt_id = cur.lastrowid
    return f"Appointment booked! ID: {appt_id}. {patient_name} with {doctor} at {slot}. We'll send a reminder before the visit."

cur.fetchone() => returns the next row - if no rows then returns None

cur.lastrowid => gives the auto generated PK ID

In [8]:
def list_appointments(phone: Optional[str] = None) -> str:
    """
    List appointments. If phone is provided, returns appointments for that phone number only.
    """
    cur = conn.cursor()
    if phone:
        cur.execute("SELECT id, patient_name, doctor, datetime, status FROM appointments WHERE phone=? ORDER BY datetime", (phone,))
    else:
        cur.execute("SELECT id, patient_name, doctor, datetime, status FROM appointments ORDER BY datetime")
    rows = cur.fetchall()
    if not rows:
        return "No appointments found."
    lines = []
    for r in rows:
        lines.append(f"ID:{r[0]} | {r[1]} | {r[2]} | {r[3]} | {r[4]}")
    return "\n".join(lines)

In [9]:
def cancel_appointment(appointment_id: Optional[int] = None, phone: Optional[str] = None,
                       date: Optional[str] = None, time: Optional[str] = None) -> str:
    """
    Cancel an appointment either by appointment_id OR by (phone + date + time).
    Date: YYYY-MM-DD, Time: HH:MM
    """
    cur = conn.cursor()
    if appointment_id:
        cur.execute("SELECT status FROM appointments WHERE id=?", (appointment_id,))
        r = cur.fetchone()
        if not r:
            return f"No appointment with ID {appointment_id}."
        if r[0] == "cancelled":
            return f"Appointment {appointment_id} already cancelled."
        cur.execute("UPDATE appointments SET status='cancelled' WHERE id=?", (appointment_id,))
        conn.commit()
        return f"Appointment {appointment_id} cancelled."

    if phone and date and time:
        slot = _make_slot_str(date, time)
        cur.execute("SELECT id FROM appointments WHERE phone=? AND datetime=? AND status='booked'", (phone, slot))
        r = cur.fetchone()
        if not r:
            return f"No matching booked appointment found for {phone} at {slot}."
        cur.execute("UPDATE appointments SET status='cancelled' WHERE id=?", (r[0],))
        conn.commit()
        return f"Appointment {r[0]} cancelled."

    return "Please provide an appointment_id or (phone + date + time) to cancel."

In [10]:
from autogen_agentchat.ui import Console

model_client = OpenAIChatCompletionClient(model="gpt-4.1-nano")
agent = AssistantAgent(
    max_tool_iterations=5,
    name="helpdesk",
    model_client=model_client,
    system_message=helpdesk_bot_system_message,
    tools=[check_availability, book_appointment, list_appointments],
    reflect_on_tool_use=True,
)
await Console(
    agent.run_stream(task="I want to book an appointment. But what are the slots available currently for 20th september 2025"),
    output_stats=True,  # Enable stats printing.
)

---------- TextMessage (user) ----------
I want to book an appointment. But what are the slots available currently for 20th september 2025
---------- ThoughtEvent (helpdesk) ----------
Thank you for reaching out. Let me check the available time slots for 20th September 2025. Please hold on for a moment.
---------- ToolCallRequestEvent (helpdesk) ----------
[FunctionCall(id='call_2k6c1ODn5VDgnfTMuZ8HRPyk', arguments='{"date":"2025-09-20"}', name='check_availability')]
[Prompt tokens: 508, Completion tokens: 50]
---------- ToolCallExecutionEvent (helpdesk) ----------
[FunctionExecutionResult(content='Available slots for Dr. Ravish Mehta on 2025-09-20: 09:00, 09:30, 10:00, 10:30, 11:00, 11:30, 12:30, 13:00, 13:30, 14:00, 14:30, 15:00, 15:30, 16:00, 16:30', name='check_availability', call_id='call_2k6c1ODn5VDgnfTMuZ8HRPyk', is_error=False)]
---------- TextMessage (helpdesk) ----------
The available slots for 20th September 2025 are from 09:00 to 16:30, with 30-minute intervals. Please let 

TaskResult(messages=[TextMessage(id='bdb543f5-a477-4f1e-b057-d643ba640be4', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 20, 9, 16, 23, 14217, tzinfo=datetime.timezone.utc), content='I want to book an appointment. But what are the slots available currently for 20th september 2025', type='TextMessage'), ThoughtEvent(id='5455b639-a749-49da-afb8-577b9017a2a4', source='helpdesk', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 20, 9, 16, 25, 50988, tzinfo=datetime.timezone.utc), content='Thank you for reaching out. Let me check the available time slots for 20th September 2025. Please hold on for a moment.', type='ThoughtEvent'), ToolCallRequestEvent(id='69f7bd91-86d2-4fe0-ba16-e00404385304', source='helpdesk', models_usage=RequestUsage(prompt_tokens=508, completion_tokens=50), metadata={}, created_at=datetime.datetime(2025, 9, 20, 9, 16, 25, 56076, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_2k6c1ODn5VDgnfTMuZ8

In [12]:
await Console(
    agent.run_stream(task="I would like to book a 12 pm appointment. Name is Chaitanya Mangesh Gokhale, number is 8007437334"),
    output_stats=True,  # Enable stats printing.
)

---------- TextMessage (user) ----------
I would like to book a 12 pm appointment. Name is Chaitanya Mangesh Gokhale, number is 8007437334
---------- ToolCallRequestEvent (helpdesk) ----------
[FunctionCall(id='call_C1xzxXdyHaPe6VTY4d04I3Co', arguments='{"patient_name": "Chaitanya Mangesh Gokhale", "phone": "8007437334", "date": "2025-09-20", "time": "12:00"}', name='book_appointment'), FunctionCall(id='call_O8CjSmqpkJZLvvC0loHXi1Xr', arguments='{"date": "2025-09-20"}', name='check_availability')]
[Prompt tokens: 752, Completion tokens: 82]
---------- ToolCallExecutionEvent (helpdesk) ----------
[FunctionExecutionResult(content='Sorry, Dr. Ravish Mehta already has an appointment at 2025-09-20 12:00. Please choose another slot.', name='book_appointment', call_id='call_C1xzxXdyHaPe6VTY4d04I3Co', is_error=False), FunctionExecutionResult(content='Available slots for Dr. Ravish Mehta on 2025-09-20: 09:00, 09:30, 10:00, 10:30, 11:00, 11:30, 12:30, 13:00, 13:30, 14:00, 14:30, 15:00, 15:30, 16

TaskResult(messages=[TextMessage(id='d74531e2-e34e-44cf-8b2f-a540254ad246', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 20, 9, 18, 34, 421663, tzinfo=datetime.timezone.utc), content='I would like to book a 12 pm appointment. Name is Chaitanya Mangesh Gokhale, number is 8007437334', type='TextMessage'), ToolCallRequestEvent(id='eea2b099-7eea-48f0-a89c-9852711035a0', source='helpdesk', models_usage=RequestUsage(prompt_tokens=752, completion_tokens=82), metadata={}, created_at=datetime.datetime(2025, 9, 20, 9, 18, 36, 491524, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_C1xzxXdyHaPe6VTY4d04I3Co', arguments='{"patient_name": "Chaitanya Mangesh Gokhale", "phone": "8007437334", "date": "2025-09-20", "time": "12:00"}', name='book_appointment'), FunctionCall(id='call_O8CjSmqpkJZLvvC0loHXi1Xr', arguments='{"date": "2025-09-20"}', name='check_availability')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='1b752eb0-1289-4275-b4c4

In [13]:
await Console(
    agent.run_stream(task="That would be fine. book it at the next one."),
    output_stats=True,  # Enable stats printing.
)

---------- TextMessage (user) ----------
That would be fine. book it at the next one.
---------- ToolCallRequestEvent (helpdesk) ----------
[FunctionCall(id='call_5DF2tAUH7oXy5QHW92ACEF65', arguments='{"patient_name":"Chaitanya Mangesh Gokhale","phone":"8007437334","date":"2025-09-20","time":"12:30"}', name='book_appointment')]
[Prompt tokens: 1038, Completion tokens: 46]
---------- ToolCallExecutionEvent (helpdesk) ----------
[FunctionExecutionResult(content="Appointment booked! ID: 3. Chaitanya Mangesh Gokhale with Dr. Ravish Mehta at 2025-09-20 12:30. We'll send a reminder before the visit.", name='book_appointment', call_id='call_5DF2tAUH7oXy5QHW92ACEF65', is_error=False)]
---------- TextMessage (helpdesk) ----------
Your appointment has been successfully booked for 12:30 PM on 20th September 2025 with Dr. Ravish Mehta. If you need any further assistance, please feel free to ask.
[Prompt tokens: 1138, Completion tokens: 42]
---------- Summary ----------
Number of messages: 4
Finish

TaskResult(messages=[TextMessage(id='68315c6b-98f3-4ceb-963c-113cb348a84c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 20, 9, 19, 1, 732444, tzinfo=datetime.timezone.utc), content='That would be fine. book it at the next one.', type='TextMessage'), ToolCallRequestEvent(id='fd6819d6-0f3b-4a1d-9d8d-098541417b66', source='helpdesk', models_usage=RequestUsage(prompt_tokens=1038, completion_tokens=46), metadata={}, created_at=datetime.datetime(2025, 9, 20, 9, 19, 2, 831162, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_5DF2tAUH7oXy5QHW92ACEF65', arguments='{"patient_name":"Chaitanya Mangesh Gokhale","phone":"8007437334","date":"2025-09-20","time":"12:30"}', name='book_appointment')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='10469b85-337e-4acc-87cf-68559a5caede', source='helpdesk', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 20, 9, 19, 2, 852755, tzinfo=datetime.timezone.utc), content=[Functio

In [14]:
await Console(
    agent.run_stream(task="List all appointments for 20th september."),
    output_stats=True,  # Enable stats printing.
)

---------- TextMessage (user) ----------
List all appointments for 20th september.
---------- ToolCallRequestEvent (helpdesk) ----------
[FunctionCall(id='call_xqi3TfaeYMJL3YoZWJwoQiNa', arguments='{"phone":"8007437334"}', name='list_appointments')]
[Prompt tokens: 1197, Completion tokens: 18]
---------- ToolCallExecutionEvent (helpdesk) ----------
[FunctionExecutionResult(content='ID:2 | Chaitanya Mangesh Gokhale | Dr. Ravish Mehta | 2025-09-20 12:00 | booked\nID:3 | Chaitanya Mangesh Gokhale | Dr. Ravish Mehta | 2025-09-20 12:30 | booked', name='list_appointments', call_id='call_xqi3TfaeYMJL3YoZWJwoQiNa', is_error=False)]
---------- TextMessage (helpdesk) ----------
Here are the appointments for 20th September 2025:
1. Chaitanya Mangesh Gokhale with Dr. Ravish Mehta at 12:00 PM
2. Chaitanya Mangesh Gokhale with Dr. Ravish Mehta at 12:30 PM

If you need to make any changes or have any other questions, please let me know.
[Prompt tokens: 1293, Completion tokens: 82]
---------- Summary 

TaskResult(messages=[TextMessage(id='b9916792-e1cd-479e-8534-32ae71a326b3', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 20, 9, 20, 47, 928783, tzinfo=datetime.timezone.utc), content='List all appointments for 20th september.', type='TextMessage'), ToolCallRequestEvent(id='ffad8b14-45eb-4572-9a5d-769fae52adcb', source='helpdesk', models_usage=RequestUsage(prompt_tokens=1197, completion_tokens=18), metadata={}, created_at=datetime.datetime(2025, 9, 20, 9, 20, 49, 76993, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_xqi3TfaeYMJL3YoZWJwoQiNa', arguments='{"phone":"8007437334"}', name='list_appointments')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='2577b95a-7557-4b2c-8e77-71336343d405', source='helpdesk', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 20, 9, 20, 49, 81887, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='ID:2 | Chaitanya Mangesh Gokhale | Dr. Ravish Mehta |